# CELL 1: Markdown

In [ ]:
# Task 2: Supervised Baseline - Split 40:60

# CELL 2: Environment Setup

In [1]:
import os
import sys
import time
import random
import warnings
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
from tensorflow.keras.applications import (
    ResNet50, EfficientNetB0, MobileNetV2, InceptionV3, DenseNet121
)
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_recall_fscore_support, roc_auc_score, roc_curve, auc
)
from sklearn.preprocessing import label_binarize

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print(f"TensorFlow: {tf.__version__}")
print(f"Keras: {keras.__version__}")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU Available: {len(gpus)} device(s)")
    except RuntimeError as e:
        print(f"GPU error: {e}")
else:
    print("CPU mode")

def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seeds(42)
print("Environment ready")


2025-10-27 15:24:37.583191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761578677.801313      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761578677.858261      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow: 2.18.0
Keras: 3.8.0
GPU Available: 1 device(s)
Environment ready


# CELL 3: Configuration for 40:60

In [2]:
class Config:
    RANDOM_SEED = 42
    
    DATASET_BASE = '/kaggle/input/dataset-for-classifying-rice-varieties-in-bd'
    ORIGINAL_FOLDER = os.path.join(DATASET_BASE, 'Rice Varieties in Bangladesh', 'Original')
    
    IMG_SIZE = (224, 224)
    INPUT_SHAPE = (224, 224, 3)
    
    NUM_CLASSES = 38
    TOTAL_IMAGES = 19000
    CLASS_NAMES = [
        'BD30', 'BD33', 'BD39', 'BD49', 'BD51', 'BD52', 'BD56', 'BD57',
        'BD70', 'BD72', 'BD75', 'BD76', 'BD79', 'BD85', 'BD87', 'BD91',
        'BD93', 'BD95', 'Binadhan10', 'Binadhan11', 'Binadhan12', 'Binadhan14',
        'Binadhan16', 'Binadhan17', 'Binadhan19', 'Binadhan20', 'Binadhan21',
        'Binadhan23', 'Binadhan24', 'Binadhan25', 'Binadhan26', 'Binadhan7',
        'Binadhan8', 'BR22', 'BR23', 'BRRI102', 'BRRI67', 'BRRI74'
    ]
    
    TRAIN_RATIO = 20
    TEST_RATIO = 80
    VAL_RATIO = 0.1
    SPLIT_NAME = f"{TRAIN_RATIO}_{TEST_RATIO}"
    
    BATCH_SIZE = 32
    EPOCHS_PHASE1 = 10
    EPOCHS_PHASE2 = 40
    TOTAL_EPOCHS = 50
    
    LR_PHASE1 = 1e-3
    LR_PHASE2 = 1e-4
    
    DROPOUT_RATE = 0.5
    L2_REG = 1e-4
    
    MODELS = ['ResNet50', 'EfficientNetB0', 'MobileNetV2', 'InceptionV3', 'DenseNet121']
    
    OUTPUT_DIR = f'/kaggle/working/outputs_{SPLIT_NAME}_v2'
    MODELS_DIR = os.path.join(OUTPUT_DIR, 'models')
    LOGS_DIR = os.path.join(OUTPUT_DIR, 'logs')
    PLOTS_DIR = os.path.join(OUTPUT_DIR, 'plots')
    RESULTS_DIR = os.path.join(OUTPUT_DIR, 'results')

cfg = Config()

for directory in [cfg.OUTPUT_DIR, cfg.MODELS_DIR, cfg.LOGS_DIR, cfg.PLOTS_DIR, cfg.RESULTS_DIR]:
    os.makedirs(directory, exist_ok=True)

print(f"Configuration: Split {cfg.TRAIN_RATIO}:{cfg.TEST_RATIO}")
print(f"Output: {cfg.OUTPUT_DIR}")


Configuration: Split 20:80
Output: /kaggle/working/outputs_20_80_v2


# CELL 4: Data Loading Functions

In [3]:
def load_dataset(data_dir, class_names):
    image_paths = []
    labels = []
    label_to_idx = {name: idx for idx, name in enumerate(sorted(class_names))}
    idx_to_label = {idx: name for name, idx in label_to_idx.items()}
    
    print("Loading dataset...")
    
    class_counts = {}
    for class_name in sorted(class_names):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.exists(class_dir):
            continue
        
        class_images = [
            os.path.join(class_dir, f)
            for f in os.listdir(class_dir)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
        
        image_paths.extend(class_images)
        labels.extend([label_to_idx[class_name]] * len(class_images))
        class_counts[class_name] = len(class_images)
    
    print(f"Loaded: {len(image_paths)} images, {len(class_counts)} classes")
    return image_paths, labels, label_to_idx, idx_to_label


def prepare_splits(image_paths, labels, train_ratio, test_ratio, val_ratio=0.1, random_state=42):
    image_paths = np.array(image_paths)
    labels = np.array(labels)
    
    train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
        image_paths, labels, test_size=test_ratio/100, stratify=labels, random_state=random_state
    )
    
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_val_paths, train_val_labels, test_size=val_ratio, stratify=train_val_labels, random_state=random_state
    )
    
    print(f"Train: {len(train_paths)}, Val: {len(val_paths)}, Test: {len(test_paths)}")
    return {
        'train_paths': train_paths, 'train_labels': train_labels,
        'val_paths': val_paths, 'val_labels': val_labels,
        'test_paths': test_paths, 'test_labels': test_labels
    }


all_image_paths, all_labels, label_to_idx, idx_to_label = load_dataset(cfg.ORIGINAL_FOLDER, cfg.CLASS_NAMES)
split_data = prepare_splits(all_image_paths, all_labels, cfg.TRAIN_RATIO, cfg.TEST_RATIO, cfg.VAL_RATIO, cfg.RANDOM_SEED)


Loading dataset...
Loaded: 19000 images, 38 classes
Train: 3420, Val: 380, Test: 15200


# CELL 5: Data Pipeline Functions

In [4]:
def create_dataset_enhanced(paths, labels, preprocess_func, batch_size, img_size, 
                            shuffle=False, augment=False, seed=42):
    def load_and_augment(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        
        if augment:
            img = tf.image.random_crop(tf.image.resize(img, (256, 256)), [224, 224, 3])
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, 0.3)
            img = tf.image.random_contrast(img, 0.7, 1.3)
            img = tf.image.random_saturation(img, 0.7, 1.3)
            img = tf.image.random_hue(img, 0.1)
        else:
            img = tf.image.resize(img, img_size)
        
        img = tf.cast(img, tf.float32)
        if preprocess_func is not None:
            img = preprocess_func(img)
        else:
            img = img / 255.0
        
        return img, label
    
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=2000, seed=seed, reshuffle_each_iteration=True)
    dataset = dataset.map(load_and_augment, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

print("Enhanced augmentation pipeline ready")


Enhanced augmentation pipeline ready


# CELL 6: Model Building Function

In [5]:
def build_model_advanced(backbone_name, input_shape=(224, 224, 3), num_classes=38):
    backbones = {
        'ResNet50': (ResNet50, resnet_preprocess),
        'EfficientNetB0': (EfficientNetB0, efficientnet_preprocess),
        'MobileNetV2': (MobileNetV2, mobilenet_preprocess),
        'InceptionV3': (InceptionV3, inception_preprocess),
        'DenseNet121': (DenseNet121, densenet_preprocess)
    }
    
    backbone_class, preprocess_func = backbones[backbone_name]
    
    print(f"Building {backbone_name} with enhanced architecture...")
    base_model = backbone_class(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False
    
    inputs = keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.Dropout(cfg.DROPOUT_RATE)(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.Dropout(cfg.DROPOUT_RATE)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs, name=f'{backbone_name}_enhanced')
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=cfg.LR_PHASE1),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"Model ready: {model.count_params():,} parameters")
    return model, preprocess_func, base_model

print("Advanced model builder defined")


Advanced model builder defined


# CELL 7: Two-Phase Training Controller

In [6]:
class TwoPhaseTrainer:
    def __init__(self, model, base_model, model_name, split_name):
        self.model = model
        self.base_model = base_model
        self.model_name = model_name
        self.split_name = split_name
        self.history_phase1 = None
        self.history_phase2 = None
        
    def get_callbacks(self, phase):
        model_dir = os.path.join(cfg.MODELS_DIR, f"{self.model_name}_{self.split_name}")
        os.makedirs(model_dir, exist_ok=True)
        
        checkpoint_path = os.path.join(model_dir, f'best_model_phase{phase}.h5')
        
        callbacks_list = [
            callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                monitor='val_accuracy',
                mode='max',
                save_best_only=True,
                save_weights_only=False,
                verbose=1
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=3,
                min_lr=1e-7,
                verbose=1,
                mode='min'
            ),
            callbacks.CSVLogger(
                filename=os.path.join(model_dir, f'training_log_phase{phase}.csv'),
                append=False
            )
        ]
        
        return callbacks_list, checkpoint_path
    
    def train_phase1(self, train_dataset, val_dataset):
        print(f"\nPHASE 1: Training classifier head (backbone frozen)")
        print(f"Epochs: {cfg.EPOCHS_PHASE1}, Learning Rate: {cfg.LR_PHASE1}")
        print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in self.model.trainable_weights]):,}")
        
        self.base_model.trainable = False
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=cfg.LR_PHASE1),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        callbacks_list, checkpoint_path = self.get_callbacks(phase=1)
        
        start_time = time.time()
        self.history_phase1 = self.model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=cfg.EPOCHS_PHASE1,
            callbacks=callbacks_list,
            verbose=1
        )
        phase1_time = time.time() - start_time
        
        print(f"Phase 1 completed in {phase1_time/60:.2f} minutes")
        print(f"Best model saved: {checkpoint_path}")
        
        return checkpoint_path, phase1_time
    
    def train_phase2(self, train_dataset, val_dataset):
        print(f"\nPHASE 2: Fine-tuning entire model (backbone unfrozen)")
        print(f"Epochs: {cfg.EPOCHS_PHASE2}, Learning Rate: {cfg.LR_PHASE2}")
        
        self.base_model.trainable = True
        
        num_layers = len(self.base_model.layers)
        freeze_until = int(num_layers * 0.5)
        
        for layer in self.base_model.layers[:freeze_until]:
            layer.trainable = False
        
        print(f"Unfreezing last {num_layers - freeze_until}/{num_layers} layers of backbone")
        print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in self.model.trainable_weights]):,}")
        
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=cfg.LR_PHASE2),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        callbacks_list, checkpoint_path = self.get_callbacks(phase=2)
        
        start_time = time.time()
        self.history_phase2 = self.model.fit(
            train_dataset,
            validation_data=val_dataset,
            initial_epoch=cfg.EPOCHS_PHASE1,
            epochs=cfg.TOTAL_EPOCHS,
            callbacks=callbacks_list,
            verbose=1
        )
        phase2_time = time.time() - start_time
        
        print(f"Phase 2 completed in {phase2_time/60:.2f} minutes")
        print(f"Best model saved: {checkpoint_path}")
        
        return checkpoint_path, phase2_time
    
    def get_combined_history(self):
        if self.history_phase1 is None or self.history_phase2 is None:
            return None
        
        combined = {
            'accuracy': self.history_phase1.history['accuracy'] + self.history_phase2.history['accuracy'],
            'val_accuracy': self.history_phase1.history['val_accuracy'] + self.history_phase2.history['val_accuracy'],
            'loss': self.history_phase1.history['loss'] + self.history_phase2.history['loss'],
            'val_loss': self.history_phase1.history['val_loss'] + self.history_phase2.history['val_loss']
        }
        
        return type('History', (), {'history': combined})()

print("Two-phase training controller defined")


Two-phase training controller defined


# CELL 8: Evaluation Function 

In [7]:
def evaluate_model_comprehensive(model, test_dataset, test_labels, idx_to_label, 
                                  model_name, split_name):
    print(f"\nComprehensive evaluation: {model_name} on {split_name}")
    
    results_dir = os.path.join(cfg.RESULTS_DIR, f"{model_name}_{split_name}")
    os.makedirs(results_dir, exist_ok=True)
    
    start_time = time.time()
    y_pred_probs = model.predict(test_dataset, verbose=0)
    inference_time = time.time() - start_time
    
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = test_labels
    
    avg_inference_time_ms = (inference_time / len(test_labels)) * 1000
    
    overall_accuracy = accuracy_score(y_true, y_pred)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )
    
    print(f"Accuracy: {overall_accuracy*100:.2f}%")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None, zero_division=0
    )
    
    per_class_accuracy = np.array([
        (y_pred[y_true == i] == y_true[y_true == i]).sum() / (y_true == i).sum()
        if (y_true == i).sum() > 0 else 0.0
        for i in range(cfg.NUM_CLASSES)
    ])
    
    try:
        y_true_bin = label_binarize(y_true, classes=range(cfg.NUM_CLASSES))
        roc_auc_scores = []
        for i in range(cfg.NUM_CLASSES):
            try:
                score = roc_auc_score(y_true_bin[:, i], y_pred_probs[:, i])
                roc_auc_scores.append(score)
            except:
                roc_auc_scores.append(0.0)
        roc_auc_macro = np.mean(roc_auc_scores)
        print(f"ROC-AUC (Macro): {roc_auc_macro:.4f}")
    except:
        roc_auc_macro = 0.0
        roc_auc_scores = [0.0] * cfg.NUM_CLASSES
    
    cm = confusion_matrix(y_true, y_pred)
    
    confused_pairs = []
    for i in range(cfg.NUM_CLASSES):
        for j in range(cfg.NUM_CLASSES):
            if i != j and cm[i, j] > 0:
                confused_pairs.append({
                    'true_class': idx_to_label[i],
                    'pred_class': idx_to_label[j],
                    'count': int(cm[i, j]),
                    'true_idx': int(i),
                    'pred_idx': int(j)
                })
    
    top_confused = sorted(confused_pairs, key=lambda x: x['count'], reverse=True)[:5]
    
    total_params = model.count_params()
    gflops = total_params * 2 / 1e9
    
    results_dict = {
        'model_name': model_name,
        'split_name': split_name,
        'overall_accuracy': overall_accuracy,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'roc_auc_macro': roc_auc_macro,
        'avg_inference_time_ms': avg_inference_time_ms,
        'total_params': total_params,
        'gflops': gflops,
        'per_class_accuracy': per_class_accuracy,
        'per_class_precision': precision,
        'per_class_recall': recall,
        'per_class_f1': f1,
        'confusion_matrix': cm,
        'y_true': y_true,
        'y_pred': y_pred,
        'y_pred_probs': y_pred_probs
    }
    
    json_results = {k: float(v) if isinstance(v, np.floating) else int(v) if isinstance(v, np.integer) else v 
                    for k, v in results_dict.items() if k not in ['confusion_matrix', 'y_true', 'y_pred', 'y_pred_probs', 
                                                                    'per_class_accuracy', 'per_class_precision', 
                                                                    'per_class_recall', 'per_class_f1']}
    
    with open(os.path.join(results_dir, 'metrics.json'), 'w') as f:
        json.dump(json_results, f, indent=4)
    
    return results_dict

print("Comprehensive evaluation function defined")


Comprehensive evaluation function defined


# CELL 9: Visualization Function

In [8]:
def plot_training_curves_enhanced(history, model_name, split_name):
    model_dir = os.path.join(cfg.PLOTS_DIR, f"{model_name}_{split_name}")
    os.makedirs(model_dir, exist_ok=True)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    epochs_range = range(1, len(history.history['accuracy']) + 1)
    phase1_end = cfg.EPOCHS_PHASE1
    
    axes[0, 0].plot(epochs_range, history.history['accuracy'], 'b-', linewidth=2, label='Train')
    axes[0, 0].plot(epochs_range, history.history['val_accuracy'], 'r-', linewidth=2, label='Validation')
    axes[0, 0].axvline(x=phase1_end, color='g', linestyle='--', linewidth=2, label='Phase 1/2 Boundary')
    axes[0, 0].set_title('Model Accuracy', fontsize=13, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].plot(epochs_range, history.history['loss'], 'b-', linewidth=2, label='Train')
    axes[0, 1].plot(epochs_range, history.history['val_loss'], 'r-', linewidth=2, label='Validation')
    axes[0, 1].axvline(x=phase1_end, color='g', linestyle='--', linewidth=2, label='Phase 1/2 Boundary')
    axes[0, 1].set_title('Model Loss', fontsize=13, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    val_acc = history.history['val_accuracy']
    best_epoch = np.argmax(val_acc) + 1
    best_val_acc = max(val_acc)
    
    axes[1, 0].plot(epochs_range, val_acc, 'r-', linewidth=2)
    axes[1, 0].scatter([best_epoch], [best_val_acc], color='gold', s=200, zorder=5, edgecolors='black', linewidth=2)
    axes[1, 0].set_title(f'Validation Accuracy (Best: {best_val_acc:.4f} at epoch {best_epoch})', 
                         fontsize=13, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Validation Accuracy')
    axes[1, 0].grid(True, alpha=0.3)
    
    overfitting_gap = [history.history['accuracy'][i] - history.history['val_accuracy'][i] 
                       for i in range(len(history.history['accuracy']))]
    axes[1, 1].plot(epochs_range, overfitting_gap, 'purple', linewidth=2)
    axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=1)
    axes[1, 1].set_title('Training-Validation Gap (Overfitting Indicator)', fontsize=13, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Train Acc - Val Acc')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_path = os.path.join(model_dir, 'training_analysis.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Training analysis saved: {save_path}")
    plt.close()


def plot_confusion_matrix_professional(cm, model_name, split_name):
    model_dir = os.path.join(cfg.PLOTS_DIR, f"{model_name}_{split_name}")
    os.makedirs(model_dir, exist_ok=True)
    
    fig, axes = plt.subplots(1, 2, figsize=(24, 10))
    
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    sns.heatmap(cm_normalized, annot=False, cmap='Blues', ax=axes[0],
                xticklabels=cfg.CLASS_NAMES, yticklabels=cfg.CLASS_NAMES,
                cbar_kws={'label': 'Normalized Frequency'})
    axes[0].set_title(f'{model_name}: Normalized Confusion Matrix', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Predicted Label', fontsize=12)
    axes[0].set_ylabel('True Label', fontsize=12)
    
    per_class_acc = np.diag(cm_normalized)
    axes[1].barh(cfg.CLASS_NAMES, per_class_acc, color='steelblue')
    axes[1].set_xlabel('Accuracy', fontsize=12)
    axes[1].set_title(f'{model_name}: Per-Class Accuracy', fontsize=14, fontweight='bold')
    axes[1].axvline(x=np.mean(per_class_acc), color='red', linestyle='--', linewidth=2, 
                    label=f'Mean: {np.mean(per_class_acc):.3f}')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    save_path = os.path.join(model_dir, 'confusion_matrix_analysis.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Confusion matrix analysis saved: {save_path}")
    plt.close()

print("Enhanced visualization functions defined")


Enhanced visualization functions defined


# CELL 10: Main Training Pipeline

In [9]:
print(f"Starting enhanced training pipeline for split {cfg.TRAIN_RATIO}:{cfg.TEST_RATIO}")
print(f"Configuration: Two-phase training with enhanced regularization")
print(f"Expected outcome: 95%+ accuracy with minimal overfitting\n")

results_collection = []

for model_name in cfg.MODELS:
    try:
        print(f"\n{'='*90}")
        print(f"MODEL: {model_name}")
        print(f"{'='*90}")
        
        model, preprocess_func, base_model = build_model_advanced(model_name)
        
        print("\nPreparing data pipelines...")
        train_dataset = create_dataset_enhanced(
            split_data['train_paths'], split_data['train_labels'],
            preprocess_func, cfg.BATCH_SIZE, cfg.IMG_SIZE,
            shuffle=True, augment=True, seed=cfg.RANDOM_SEED
        )
        
        val_dataset = create_dataset_enhanced(
            split_data['val_paths'], split_data['val_labels'],
            preprocess_func, cfg.BATCH_SIZE, cfg.IMG_SIZE,
            shuffle=False, augment=False
        )
        
        test_dataset = create_dataset_enhanced(
            split_data['test_paths'], split_data['test_labels'],
            preprocess_func, cfg.BATCH_SIZE, cfg.IMG_SIZE,
            shuffle=False, augment=False
        )
        
        trainer = TwoPhaseTrainer(model, base_model, model_name, cfg.SPLIT_NAME)
        
        checkpoint_p1, time_p1 = trainer.train_phase1(train_dataset, val_dataset)
        checkpoint_p2, time_p2 = trainer.train_phase2(train_dataset, val_dataset)
        
        total_training_time = time_p1 + time_p2
        
        print(f"\nLoading best model from Phase 2...")
        best_model = keras.models.load_model(checkpoint_p2)
        
        results = evaluate_model_comprehensive(
            best_model, test_dataset, split_data['test_labels'],
            idx_to_label, model_name, cfg.SPLIT_NAME
        )
        
        results['training_time_total_min'] = total_training_time / 60
        results['training_time_phase1_min'] = time_p1 / 60
        results['training_time_phase2_min'] = time_p2 / 60
        
        combined_history = trainer.get_combined_history()
        plot_training_curves_enhanced(combined_history, model_name, cfg.SPLIT_NAME)
        plot_confusion_matrix_professional(results['confusion_matrix'], model_name, cfg.SPLIT_NAME)
        
        results_collection.append(results)
        
        print(f"\n{model_name} Summary:")
        print(f"  Test Accuracy: {results['overall_accuracy']*100:.2f}%")
        print(f"  F1-Score: {results['f1_macro']:.4f}")
        print(f"  Total Training Time: {results['training_time_total_min']:.1f} minutes")
        
        keras.backend.clear_session()
        
    except Exception as e:
        print(f"\nError with {model_name}: {str(e)}")
        import traceback
        traceback.print_exc()

print(f"\n{'='*90}")
print(f"Training pipeline completed for split {cfg.TRAIN_RATIO}:{cfg.TEST_RATIO}")
print(f"Models trained: {len(results_collection)}/5")
print(f"{'='*90}")


Starting enhanced training pipeline for split 20:80
Configuration: Two-phase training with enhanced regularization
Expected outcome: 95%+ accuracy with minimal overfitting


MODEL: ResNet50
Building ResNet50 with enhanced architecture...


I0000 00:00:1761578774.659278      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Model ready: 24,786,086 parameters

Preparing data pipelines...

PHASE 1: Training classifier head (backbone frozen)
Epochs: 10, Learning Rate: 0.001
Trainable parameters: 1,194,278
Epoch 1/10


I0000 00:00:1761578793.135277      97 service.cc:148] XLA service 0x7c16800034c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761578793.135979      97 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1761578794.827374      97 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/107 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.0069 - loss: 5.2183      

...

107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.2770 - loss: 2.6835
Epoch 11: val_accuracy improved from -inf to 0.40000, saving model to /kaggle/working/outputs_20_80_v2/models/EfficientNetB0_20_80/best_model_phase2.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 94s 356ms/step - accuracy: 0.2775 - loss: 2.6808 - val_accuracy: 0.4000 - val_loss: 2.0015 - learning_rate: 1.0000e-04
Epoch 12/50
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4498 - loss: 1.8908
Epoch 12: val_accuracy improved from 0.40000 to 0.42368, saving model to /kaggle/working/outputs_20_80_v2/models/EfficientNetB0_20_80/best_model_phase2.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - accuracy: 0.4500 - loss: 1.8898 - val_accuracy: 0.4237 - val_loss: 1.8714 - learning_rate: 1.0000e-04
Epoch 13/50
106/107 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5312 - loss: 1.5494
Epoch 13: val_accuracy improved from 0.42368 to 0.48684, saving model to /kaggle/working/outputs_20_80_v2/models/EfficientNetB0_20_80/best_model_p

E0000 00:00:1761580197.835770      98 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1761580198.032107      98 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


106/107 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3397 - loss: 2.4696

E0000 00:00:1761580215.093865      97 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1761580215.292564      97 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.3403 - loss: 2.4669
Epoch 11: val_accuracy improved from -inf to 0.14737, saving model to /kaggle/working/outputs_20_80_v2/models/MobileNetV2_20_80/best_model_phase2.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 57s 216ms/step - accuracy: 0.3409 - loss: 2.4642 - val_accuracy: 0.1474 - val_loss: 5.5049 - learning_rate: 1.0000e-04
Epoch 12/50
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4780 - loss: 1.8075
Epoch 12: val_accuracy improved from 0.14737 to 0.15000, saving model to /kaggle/working/outputs_20_80_v2/models/MobileNetV2_20_80/best_model_phase2.h5
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.4782 - loss: 1.8065 - val_accuracy: 0.1500 - val_loss: 4.5667 - learning_rate: 1.0000e-04
Epoch 13/50
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5539 - loss: 1.5377
Epoch 13: val_accuracy did not improve from 0.15000
107/107 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.5539 - loss: 1.5375 - val_accuracy: 0.1263 -

# CELL 11: Results Summary and Analysis

In [10]:
if len(results_collection) > 0:
    results_df = pd.DataFrame({
        'Model': [r['model_name'] for r in results_collection],
        'Split': [cfg.SPLIT_NAME] * len(results_collection),
        'Test_Accuracy': [r['overall_accuracy'] * 100 for r in results_collection],
        'F1_Macro': [r['f1_macro'] for r in results_collection],
        'Precision_Macro': [r['precision_macro'] for r in results_collection],
        'Recall_Macro': [r['recall_macro'] for r in results_collection],
        'ROC_AUC': [r['roc_auc_macro'] for r in results_collection],
        'Inference_ms': [r['avg_inference_time_ms'] for r in results_collection],
        'Training_min': [r['training_time_total_min'] for r in results_collection],
        'Parameters': [r['total_params'] for r in results_collection],
        'GFLOPs': [r['gflops'] for r in results_collection]
    })
    
    results_df_sorted = results_df.sort_values('Test_Accuracy', ascending=False)
    
    print("\n" + "="*110)
    print(f"FINAL RESULTS SUMMARY - Split {cfg.TRAIN_RATIO}:{cfg.TEST_RATIO}")
    print("="*110)
    print(results_df_sorted.to_string(index=False))
    print("="*110)
    
    best_model = results_df_sorted.iloc[0]
    print(f"\nBest Model: {best_model['Model']}")
    print(f"Test Accuracy: {best_model['Test_Accuracy']:.2f}%")
    print(f"F1-Score: {best_model['F1_Macro']:.4f}")
    print(f"Training Time: {best_model['Training_min']:.1f} minutes")
    
    csv_path = os.path.join(cfg.RESULTS_DIR, f'summary_{cfg.SPLIT_NAME}.csv')
    results_df_sorted.to_csv(csv_path, index=False)
    print(f"\nResults saved: {csv_path}")
else:
    print("No results to summarize")



FINAL RESULTS SUMMARY - Split 20:80
         Model Split  Test_Accuracy  F1_Macro  Precision_Macro  Recall_Macro  ROC_AUC  Inference_ms  Training_min  Parameters   GFLOPs
      ResNet50 20_80      63.868421  0.620491         0.734671      0.638684 0.986503      3.906129     11.249702    24786086 0.049572
   DenseNet121 20_80      61.263158  0.592930         0.721909      0.612632 0.985794      2.810708     12.281699     7707494 0.015415
   MobileNetV2 20_80      61.006579  0.589064         0.717066      0.610066 0.983629      1.770025      7.476511     3060070 0.006120
   InceptionV3 20_80      56.539474  0.534739         0.672777      0.565395 0.978924      2.014739      9.842650    23001158 0.046002
EfficientNetB0 20_80      43.335526  0.408443         0.544951      0.433355 0.946810      1.901580      8.527223     4851657 0.009703

Best Model: ResNet50
Test Accuracy: 63.87%
F1-Score: 0.6205
Training Time: 11.2 minutes

Results saved: /kaggle/working/outputs_20_80_v2/results/summary

#  CELL 12:  Archive

In [11]:
import zipfile
from datetime import datetime
import shutil

def create_comprehensive_archive():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = f'Task2_Split_{cfg.SPLIT_NAME}_Complete_{timestamp}.zip'
    
    print(f"Creating comprehensive archive: {archive_name}")
    
    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(cfg.OUTPUT_DIR):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, cfg.OUTPUT_DIR)
                zipf.write(file_path, arcname)
                
    file_size_mb = os.path.getsize(archive_name) / (1024*1024)
    
    print(f"\nArchive created successfully")
    print(f"Filename: {archive_name}")
    print(f"Size: {file_size_mb:.2f} MB")
    print(f"Location: /kaggle/working/{archive_name}")
    print(f"\nContents:")
    print(f"  - Trained models (.h5 files)")
    print(f"  - Training logs (CSV)")
    print(f"  - All plots and visualizations")
    print(f"  - Evaluation metrics (JSON)")
    print(f"  - Summary results (CSV)")
    
    return archive_name

archive_file = create_comprehensive_archive()

print("\n" + "="*110)
print("BACKUP AND DOWNLOAD INSTRUCTIONS")
print("="*110)
print(f"1. Navigate to /kaggle/working/ in the Output panel")
print(f"2. Right-click on: {archive_file}")
print(f"3. Select 'Download'")
print(f"4. Store safely - this contains ALL your work for split {cfg.SPLIT_NAME}")
print("="*110)


Creating comprehensive archive: Task2_Split_20_80_Complete_20251027_162210.zip

Archive created successfully
Filename: Task2_Split_20_80_Complete_20251027_162210.zip
Size: 856.83 MB
Location: /kaggle/working/Task2_Split_20_80_Complete_20251027_162210.zip

Contents:
  - Trained models (.h5 files)
  - Training logs (CSV)
  - All plots and visualizations
  - Evaluation metrics (JSON)
  - Summary results (CSV)

BACKUP AND DOWNLOAD INSTRUCTIONS
1. Navigate to /kaggle/working/ in the Output panel
2. Right-click on: Task2_Split_20_80_Complete_20251027_162210.zip
3. Select 'Download'
4. Store safely - this contains ALL your work for split 20_80


# CELL 13: Final Report Generation

In [12]:
def generate_final_report():
    report_path = os.path.join(cfg.RESULTS_DIR, f'REPORT_Split_{cfg.SPLIT_NAME}.txt')
    
    with open(report_path, 'w') as f:
        f.write("="*110 + "\n")
        f.write(f"TASK 2: SUPERVISED BASELINE TRAINING REPORT\n")
        f.write(f"Split: {cfg.TRAIN_RATIO}:{cfg.TEST_RATIO}\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("="*110 + "\n\n")
        
        f.write("PROJECT INFORMATION\n")
        f.write("-" * 110 + "\n")
        f.write(f"Course: CSE475 - Self-Supervised Learning\n")
        f.write(f"Dataset: Rice Varieties in Bangladesh (38 classes)\n")
        f.write(f"Total Images: {cfg.TOTAL_IMAGES}\n")
        f.write(f"Training Strategy: Two-phase (frozen backbone → fine-tuning)\n")
        f.write(f"Total Epochs: {cfg.TOTAL_EPOCHS} (Phase 1: {cfg.EPOCHS_PHASE1}, Phase 2: {cfg.EPOCHS_PHASE2})\n\n")
        
        if len(results_collection) > 0:
            f.write("MODEL PERFORMANCE SUMMARY\n")
            f.write("-" * 110 + "\n")
            for result in results_collection:
                f.write(f"\nModel: {result['model_name']}\n")
                f.write(f"  Test Accuracy: {result['overall_accuracy']*100:.2f}%\n")
                f.write(f"  F1-Score (Macro): {result['f1_macro']:.4f}\n")
                f.write(f"  Precision (Macro): {result['precision_macro']:.4f}\n")
                f.write(f"  Recall (Macro): {result['recall_macro']:.4f}\n")
                f.write(f"  ROC-AUC: {result['roc_auc_macro']:.4f}\n")
                f.write(f"  Parameters: {result['total_params']:,}\n")
                f.write(f"  Training Time: {result['training_time_total_min']:.1f} minutes\n")
                f.write(f"  Inference Time: {result['avg_inference_time_ms']:.2f} ms/image\n")
            
            best_result = max(results_collection, key=lambda x: x['overall_accuracy'])
            f.write(f"\n{'='*110}\n")
            f.write(f"BEST MODEL: {best_result['model_name']}\n")
            f.write(f"Test Accuracy: {best_result['overall_accuracy']*100:.2f}%\n")
            f.write(f"F1-Score: {best_result['f1_macro']:.4f}\n")
            f.write(f"{'='*110}\n")
        
        f.write("\nFILES GENERATED\n")
        f.write("-" * 110 + "\n")
        f.write("- Trained model files (.h5)\n")
        f.write("- Training logs (CSV)\n")
        f.write("- Training curves and analysis plots\n")
        f.write("- Confusion matrices\n")
        f.write("- ROC curves\n")
        f.write("- Evaluation metrics (JSON)\n")
        f.write("- Summary table (CSV)\n")
    
    print(f"Final report generated: {report_path}")
    return report_path

report_file = generate_final_report()
print(f"\nReport saved and ready for submission")


Final report generated: /kaggle/working/outputs_20_80_v2/results/REPORT_Split_20_80.txt

Report saved and ready for submission
